In [1]:
# import env
# # Make HTTP requests
# import requests
# # Scrape data from an HTML document
# from bs4 import BeautifulSoup
# I/O
#import os
# Search and manipulate strings
import re
#import lyricsgenius
import pandas as pd
import nltk
import draft_prepare as prep
import draft_model as model
import contractions
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix






KeyError: 'Column not found: unique_words'

In [2]:
# used to download lexicon data
# nltk.download('vader_lexicon')

In [3]:
# playing with genius API

# genius = lyricsgenius.Genius(env.token)

# artist = genius.search_artist("Andy Shauf", max_songs=3, sort="title")
# print(artist.songs)
# song = artist.songs[0]
# print(song.lyrics)
# df = pd.read_csv("charts.csv")
# df.head()
# df.shape
# #convert date to datetime
# df['date'] = pd.to_datetime(df['date'])
# df.head()
# first_date = pd.DataFrame(df.groupby(['artist', 'song']).date.min())
# # add a column for year and decade
# first_date['year'] = first_date.date.dt.year
# first_date['decade'] = first_date.date.dt.year - (first_date.date.dt.year % 10)
# first_date.reset_index(inplace=True)
# first_date.head()
# first_date.shape

In [4]:
# def prep_data(df):
#     '''
#     get unique artist song combos
#     make column for year and decade
#     change song to title
#     '''
#     # convert date to datetime
#     df['date'] = pd.to_datetime(df['date'])
#     # rename song column to title
#     df.rename(columns={'song': 'title'}, inplace=True)
#     # subset data to make only unique artist title combos while keeping earliest date
#     df = pd.DataFrame(df.groupby(['artist', 'title']).date.min())
#     # make new column for year and decade
#     df['year'] = df.date.dt.year
#     df['decade'] = df.date.dt.year - (df.date.dt.year % 10)
#     # reset index
#     df.reset_index(inplace=True)
#     return df
# df = prep_data(pd.read_csv("charts.csv"))
# df.head()
# df.shape

In [5]:
# This was used to get initial data


# api = lyricsgenius.Genius(env.token, verbose=False)

# def prep_songs():
#     songs = pd.read_csv('charts.csv')
#     songs['date'] = pd.to_datetime(songs.date)
#     songs = songs.rename(columns={'song': 'title'})
#     songs = pd.DataFrame(songs.groupby(['title', 'artist']).min().date).reset_index()
#     return songs

# def get_lyrics(title, artist):
#     song = api.search_song(title, artist)
#     lyrics = song.lyrics.replace('\n', ' ')
#     # TODO handle verse and chorus tags
#     return lyrics

# songs = prep_songs()

# for i in range(len(songs)-1, 0 , -1):
    
#     title = songs.iloc[[i]].title.values[0]
#     artist = songs.iloc[[i]].artist.values[0]
    
#     try:
#         songs.loc[i, 'lyrics'] = get_lyrics(title, artist)
#         songs.loc[i, 'status'] = 'lyrics acquired'
#     except: 
#         songs.loc[i, 'status'] = 'an error ocurred'
#     # save every 100 songs
#     if i % 100 == 0:
#         songs.to_csv('songs.csv')
    
#     print(f'{round((i / len(songs) * 100), 2)}%\t complete', end='\r')
#songs.to_csv('songs.csv')

# Prep data

In [6]:
songs = pd.read_csv('songs_0526.csv', index_col=0)
#songs.reset_index(drop = True, inplace=True)
#songs.drop(columns=['Unnamed: 0'], inplace=True)
songs.head()

,title,artist,date,lyrics
2,#1,Nelly,2001-10-20,#1 LyricsUh uh uh I just gotta bring it to the...
4,#9 Dream,John Lennon,1974-12-21,#9 Dream Lyrics[Verse 1] So long ago Was it in...
5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,..."
6,#SELFIE,The Chainsmokers,2014-03-15,#SELFIE Lyrics[Verse 1] When Jason was at the ...
7,#thatPOWER,will.i.am Featuring Justin Bieber,2013-04-06,#thatPOWER Lyrics[Instrumental break] [Pre-Ch...


In [7]:
songs.isnull().sum()

title     0
artist    0
date      0
lyrics    0
dtype: int64

In [8]:
(
songs.shape[0],
#how many lyrics contain 'Lyrics'
songs.lyrics.str.contains('Lyrics').sum(),
# how many lyrics contain 'Embed'
songs.lyrics.str.contains('Embed').sum()
)

(23762, 23762, 23762)

In [9]:
# test contractions package
contractions.fix('I\'m')


'I am'

In [10]:
songs = pd.read_csv('songs_0526.csv', index_col=0)
songs.lyrics[2] # look at raw lyrics

'#1 LyricsUh uh uh I just gotta bring it to they attention dirty, that\'s all..  You better watch who you talkin bout; runnin your mouth, like you know me You gon\' fuck around and show why the "Show Me" get called the "Show Me" Why one-on-one you can\'t hold me if your last name was Hanes Only way you wear me out is stitch my name on your pants No resident of France; but you swear I\'m from Paris Hundred-six karats - total? Naw that\'s per wrist Trying to compurr this - my chain to yo\' chain I\'m like Sprint or Motorola - no service, out of your range You out of your brains, thinkin I\'mma shout out your name You gotta come up with better ways than that to catch your fame All that pressure you applyin it\'s time to ease off Before I hit you from the blindside takin your sleeves off As much as we\'s floss, still hard to please boss Don\'t be lyin bitchin and cryin - suck it up as a loss Cause your, acts is wack, your whole label is wack And matter fact, eh eh-eh eh a-hold that [Hook] 

In [11]:
# testing for def model_clean in prepare.py

# # remove everything before and including 'lyrics'
# songs.lyrics = songs.lyrics.apply(lambda x: x.split('Lyrics')[1])
# # remove everything after and including 'Embed'
# songs.lyrics = songs.lyrics.apply(lambda x: x.rsplit('Embed')[0])
# # remove everything contained in []
# songs.lyrics = songs.lyrics.apply(lambda x: re.sub(r'\[.*?\]', '', x))
# # remove everything contained in ()
# songs.lyrics = songs.lyrics.apply(lambda x: re.sub(r'\(.*?\)', '', x))
# # expand contractions
# songs.lyrics = songs.lyrics.apply(lambda x: contractions.fix(x))

# # convert date to datetime
# songs['date'] = pd.to_datetime(songs['date'])
# # make new column for decade
# songs['decade'] = songs.date.dt.year - (songs.date.dt.year % 10)

# # songs = songs[(songs.decade != 1950) & (songs.decade != 2020)]
# songs['clean_lyrics'] = songs.lyrics.apply(prep.basic_clean)
# songs['lemmatized'] = songs.clean_lyrics.apply(prep.lemmatize)

In [12]:
songs = prep.model_clean(songs)
songs.head()

,title,artist,date,lyrics,decade
2,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,2000
4,#9 Dream,John Lennon,1974-12-21,long ago dream dream know yes know seemed real...,1970
5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,ah ah beautiful ah ah beautiful hop back bike ...,2010
6,#SELFIE,The Chainsmokers,2014-03-15,jason table kept seeing look girl think make j...,2010
7,#thatPOWER,will.i.am Featuring Justin Bieber,2013-04-06,oh alive alive alive oh fly fly fly oh alive a...,2010


In [13]:
songs.lyrics[2]

'uh uh uh got bring attention dirty better watch talkin bout runnin mouth like know gon fuck around show show get called show oneonone cannot hold last name hanes way wear stitch name pant resident france swear paris hundredsix karat total naw per wrist trying compurr chain yo chain like sprint motorola service range brain thinkin imma shout name got come better way catch fame pressure applyin time ease hit blindside takin sleeve much wes floss still hard please bos lyin bitchin cryin suck loss act wack whole label wack matter fact eh eheh eh ahold matter like take sit write number one hey hey hey hey hey hey let ask man take winner three nobody remembers take hey hey hey hey like shake ya daddy move around let get peep touch ground hell yeah love girl willin learn willin get driver seat willin turn concerned say say say think said squash probably got ebay internet access website chat line mad got mine wind flat line ohh uncle could see could see many rapper want straight emulatin styl

# modeling tryouts

In [14]:
# results = model.run_logistic_reg_models(songs)

In [15]:
# results.head(3)

In [16]:
# results.sort_values('validate_accuracy', ascending=False).head(3)

In [17]:
# results2 = model.run_decision_tree_models(songs)

In [18]:
# results2.sort_values('validate_accuracy', ascending=False).head(3)

In [108]:
results3 = model.run_random_forest_models(songs)

KeyError: 'decade'

In [ ]:
results3.sort_values('validate_accuracy', ascending=False).head(3)

# add features
~~Replace Contractions~~

~~Word Count~~

~~Unique words per song~~

~~Unique words per decade~~

~~Ratio verse to chorus~~

~~Number of verses~~

    Length of verses
    Length of chorus
    Sexual Word Count
    Vulgar Word Count

~~Sentiment~~

~~Topic Modeling~~

~~Number of verses~~ 

~~Number of choruses~~ 

~~Ratio of verses to choruses~~

~~Data/Time~~

~~year~~


In [21]:
songs = pd.read_csv('songs_0526.csv', index_col=0)
songs.head(3)


,title,artist,date,lyrics
2,#1,Nelly,2001-10-20,#1 LyricsUh uh uh I just gotta bring it to the...
4,#9 Dream,John Lennon,1974-12-21,#9 Dream Lyrics[Verse 1] So long ago Was it in...
5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,..."


In [22]:
# get all words in [] from lyrics
songs['place_words'] = songs.lyrics.apply(lambda x: re.findall(r'\[.*?\]', x))
# make place_words all lowercase
songs['place_words'] = songs.place_words.apply(lambda x: [word.lower() for word in x])
# make place_words a string
songs['place_words'] = songs.place_words.apply(lambda x: ' '.join(x))



In [23]:
songs.place_words

2                                     [hook] [hook] [hook]
4        [verse 1] [pre-chorus 1] [chorus] [verse 2] [p...
5        [intro: mariah carey] [verse 1: miguel] [choru...
6        [verse 1] [drop 1] [verse 2] [drop 2] [verse 3...
7        [instrumental break] [pre-chorus: justin biebe...
                               ...                        
28304                                                     
28305                                                     
28306    [verse 1: chad kroeger] [pre-chorus: chad kroe...
28308    [intro: will smith & (sisqo)] [verse 1: will s...
28310                                                     
Name: place_words, Length: 23762, dtype: object

In [24]:
# convert from list to string
#songs.place_words = songs.place_words.apply(lambda x: ' '.join(x))

In [25]:
songs.reset_index(inplace=True)
songs.rename(columns={'index': 'old_index'}, inplace=True)

In [26]:
#make one list from column place_words
all_place_words= ''
for i in range(len(songs)):
    all_place_words += songs.place_words[i]
# what are the counts for the common words
#all_place_words
Counter(all_place_words.split()).most_common(25)



[('[verse', 27447),
 ('[chorus]', 21833),
 ('1]', 11316),
 ('2]', 11093),
 ('[chorus:', 8000),
 ('&', 7598),
 ('[bridge]', 5635),
 ('[pre-chorus]', 5020),
 ('3]', 3836),
 ('2:', 3543),
 ('1:', 3541),
 ('[outro][verse', 2688),
 ('[chorus', 2019),
 ('[chorus][verse', 1815),
 ('3:', 1737),
 ('[outro:', 1648),
 ('[pre-chorus:', 1589),
 ('[bridge:', 1481),
 ('lil', 1220),
 ('[post-chorus]', 1192),
 ('[hook]', 1130),
 ('[outro][intro]', 958),
 ('the', 942),
 ('[hook:', 935),
 ('[refrain]', 879)]

In [27]:
# How many times is chorus in place_words?
songs.place_words.str.contains('chorus').sum()


12671

In [28]:
songs.place_words[1]

'[verse 1] [pre-chorus 1] [chorus] [verse 2] [pre-chorus 2] [chorus] [outro]'

In [29]:
test_string = songs.place_words[1] 
# precleaned
#'[verse 1] [pre-chorus 1] [chorus] [verse 2] [pre-chorus 2] [chorus] [outro]'

# how many times is chorus in test_string?
test_string.count('[chorus') # this gets multiple types of chorus without pre-chorus




2

In [30]:
test_string.count('verse')

2

In [31]:
# add column for chorus count
songs['chorus_count'] = songs.place_words.apply(lambda x: x.count('[chorus'))
# add column for verse count
songs['verse_count'] = songs.place_words.apply(lambda x: x.count('verse'))
# add column for ratio of verses to chorus
songs['verse_chorus_ratio'] = songs.verse_count / songs.chorus_count
# add column for pre-chorus count
songs['pre_chorus_count'] = songs.place_words.apply(lambda x: x.count('[pre-chorus'))
# add column for outro count
songs['outro_count'] = songs.place_words.apply(lambda x: x.count('outro'))
# add column for bridge count
songs['bridge_count'] = songs.place_words.apply(lambda x: x.count('bridge'))
# add column for hook count
songs['hook_count'] = songs.place_words.apply(lambda x: x.count('hook'))
# fill NaN in with 0
songs.verse_chorus_ratio.fillna(0, inplace=True)

In [32]:
songs.sample(3)

,old_index,title,artist,date,lyrics,place_words,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count
2656,3162,Buffalo Stance,Neneh Cherry,1989-04-01,Buffalo Stance Lyrics[Intro] Will you stop thi...,[intro] [verse 1] [interlude] [pre-chorus] [ch...,3,2,0.666667,2,1,2,0
4299,5086,Dip It Low,Christina Milian,2004-04-24,Dip It Low Lyrics[Intro: Christina Milian & Fa...,[intro: christina milian & fabolous] [verse 1:...,3,3,1.000000,0,0,0,0
4377,5175,Do It Like You,Diggy Featuring Jeremih,2012-02-11,"Do It Like You Lyrics[Chorus: Jeremih] Hey, th...",[chorus: jeremih] [verse 1: diggy] [chorus: je...,3,2,0.666667,0,1,0,0


In [33]:
df = prep.clean_df(pd.read_csv('songs_0526.csv', index_col=0))
df.head(3)

,title,artist,date,lyrics,raw_lyrics,decade
2,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,#1 LyricsUh uh uh I just gotta bring it to the...,2000
4,#9 Dream,John Lennon,1974-12-21,long ago dream dream know yes know seemed real...,#9 Dream Lyrics[Verse 1] So long ago Was it in...,1970
5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,ah ah beautiful ah ah beautiful hop back bike ...,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,...",2010


In [34]:
# df = prep.get_place_words(df)
# print("adding chorus count ********", end="\r")
# df["chorus_count"] = df.place_words.apply(lambda x: x.count("[chorus"))
# # add column for verse count
# print("adding verse count *********", end="\r")
# df["verse_count"] = df.place_words.apply(lambda x: x.count("verse"))
# # if chorus count or verse count is 0, then set ratio to 0
# df['verse_chorus_ratio'] = df.verse_count / df.chorus_count
# df.loc[(df.chorus_count == 0) | (df.verse_count == 0), 'verse_chorus_ratio'] = 0
# df['verse_chorus_ratio'] = df.verse_count / df.chorus_count

In [35]:
# def get_verse_chorus_ratio(x):
#     if x.chorus_count == 0 or x.verse_count == 0:
#         return 0
#     else:
#         return x.verse_count / x.chorus_count
        

In [36]:
# df['verse_chorus_ratio'] = df.apply(get_verse_chorus_ratio, axis=1)

In [37]:
df.describe()

,decade
count,23762.000000
mean,1984.475633
std,19.490497
min,1950.000000
25%,1970.000000
50%,1980.000000
75%,2000.000000
max,2020.000000


In [38]:
df = prep.add_features(df)
df.head(3)

Features added ******************


,title,artist,date,lyrics,raw_lyrics,decade,character_count,word_count,unique_words,unique_words_count,...,place_words,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count,bigrams,trigrams
2,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,#1 LyricsUh uh uh I just gotta bring it to the...,2000,2014,363,learn hiphop uhh album aiyyo mean catch right ...,228,...,[hook] [hook] [hook],0,0,0.0,0,0,0,3,"[(uh, uh), (uh, uh), (uh, got), (got, bring), ...","[(uh, uh, uh), (uh, uh, got), (uh, got, bring)..."
4,#9 Dream,John Lennon,1974-12-21,long ago dream dream know yes know seemed real...,#9 Dream Lyrics[Verse 1] So long ago Was it in...,1970,842,135,dancing long call air feel bowakawa go dance s...,49,...,[verse 1] [pre-chorus 1] [chorus] [verse 2] [p...,2,2,1.0,2,1,0,0,"[(long, ago), (ago, dream), (dream, dream), (d...","[(long, ago, dream), (ago, dream, dream), (dre..."
5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,ah ah beautiful ah ah beautiful hop back bike ...,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,...",2010,768,129,feel beautiful oh mean stop dance yes yeah nig...,54,...,[intro: mariah carey] [verse 1: miguel] [choru...,2,2,1.0,0,1,0,0,"[(ah, ah), (ah, beautiful), (beautiful, ah), (...","[(ah, ah, beautiful), (ah, beautiful, ah), (be..."


In [39]:
df.describe()

,decade,character_count,word_count,unique_words_count,sentiment,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count
count,23762.000000,23762.000000,23762.000000,23762.000000,23762.000000,23762.000000,23762.000000,23762.000000,23762.000000,23762.000000,23762.000000,23762.000000
mean,1984.475633,850.922355,148.609965,70.219047,0.522191,1.567124,1.495413,0.466402,0.315883,0.315335,0.331664,0.126589
std,19.490497,506.248160,89.010090,47.033070,0.746110,1.603118,1.438409,0.548411,0.799630,0.468627,0.564704,0.671409
min,1950.000000,0.000000,0.000000,0.000000,-0.999900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1970.000000,512.000000,89.000000,42.000000,0.318200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1980.000000,719.000000,124.000000,56.000000,0.962800,2.000000,2.000000,0.500000,0.000000,0.000000,0.000000,0.000000
75%,2000.000000,1026.000000,179.000000,80.000000,0.992000,3.000000,2.000000,0.750000,0.000000,1.000000,1.000000,0.000000
max,2020.000000,10282.000000,1757.000000,906.000000,0.999900,11.000000,16.000000,13.000000,6.000000,3.000000,6.000000,9.000000


In [40]:
def get_ngrams(df):
    """
    get bigrams/trigrams for each song
    """
    # create bigram column
    print("creating bigram column", end="\r")
    df["bigrams"] = df.lyrics.apply(lambda x: list(nltk.ngrams(x.split(), 2)))
    # create bigram column
    print("creating trigram column", end="\r")
    df["trigrams"] = df.lyrics.apply(lambda x: list(nltk.ngrams(x.split(), 3)))
    return df

In [41]:
df = get_ngrams(df)
df.head(3)

,title,artist,date,lyrics,raw_lyrics,decade,character_count,word_count,unique_words,unique_words_count,...,place_words,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count,bigrams,trigrams
2,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,#1 LyricsUh uh uh I just gotta bring it to the...,2000,2014,363,learn hiphop uhh album aiyyo mean catch right ...,228,...,[hook] [hook] [hook],0,0,0.0,0,0,0,3,"[(uh, uh), (uh, uh), (uh, got), (got, bring), ...","[(uh, uh, uh), (uh, uh, got), (uh, got, bring)..."
4,#9 Dream,John Lennon,1974-12-21,long ago dream dream know yes know seemed real...,#9 Dream Lyrics[Verse 1] So long ago Was it in...,1970,842,135,dancing long call air feel bowakawa go dance s...,49,...,[verse 1] [pre-chorus 1] [chorus] [verse 2] [p...,2,2,1.0,2,1,0,0,"[(long, ago), (ago, dream), (dream, dream), (d...","[(long, ago, dream), (ago, dream, dream), (dre..."
5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,ah ah beautiful ah ah beautiful hop back bike ...,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,...",2010,768,129,feel beautiful oh mean stop dance yes yeah nig...,54,...,[intro: mariah carey] [verse 1: miguel] [choru...,2,2,1.0,0,1,0,0,"[(ah, ah), (ah, beautiful), (beautiful, ah), (...","[(ah, ah, beautiful), (ah, beautiful, ah), (be..."


In [42]:
def get_unique_words(text):
    """
    Get unique words in text
    """

    # Create a list of words separated by a space
    words = text.split()
    # Create a variable to hold a list of unique words
    unique_words = set(words)
    # Join unique words list with a new string
    new_string = " ".join(unique_words)
    return new_string

In [43]:
df['unique_words'] = df.lyrics.apply(get_unique_words)
df['n_unique_words'] = df.unique_words.apply(lambda x: len(x.split()))
df.head(3)

,title,artist,date,lyrics,raw_lyrics,decade,character_count,word_count,unique_words,unique_words_count,...,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count,bigrams,trigrams,n_unique_words
2,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,#1 LyricsUh uh uh I just gotta bring it to the...,2000,2014,363,learn hiphop uhh album aiyyo mean catch right ...,228,...,0,0,0.0,0,0,0,3,"[(uh, uh), (uh, uh), (uh, got), (got, bring), ...","[(uh, uh, uh), (uh, uh, got), (uh, got, bring)...",228
4,#9 Dream,John Lennon,1974-12-21,long ago dream dream know yes know seemed real...,#9 Dream Lyrics[Verse 1] So long ago Was it in...,1970,842,135,dancing long call air feel bowakawa go dance s...,49,...,2,2,1.0,2,1,0,0,"[(long, ago), (ago, dream), (dream, dream), (d...","[(long, ago, dream), (ago, dream, dream), (dre...",49
5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,ah ah beautiful ah ah beautiful hop back bike ...,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,...",2010,768,129,feel beautiful oh mean stop dance yes yeah nig...,54,...,2,2,1.0,0,1,0,0,"[(ah, ah), (ah, beautiful), (beautiful, ah), (...","[(ah, ah, beautiful), (ah, beautiful, ah), (be...",54


In [44]:
df= prep.get_data()

Features added ******************


In [45]:
pd.set_option('max_columns', 50)
df.head(1)


,title,artist,date,lyrics,raw_lyrics,decade,character_count,word_count,unique_words,unique_words_count,sentiment,sentiment_category,place_words,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count,bigrams,trigrams
0,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,#1 LyricsUh uh uh I just gotta bring it to the...,2000,2014,363,learn hiphop uhh album aiyyo mean catch right ...,228,0.9915,very positive,[hook] [hook] [hook],0,0,0.0,0,0,0,3,"[(uh, uh), (uh, uh), (uh, got), (got, bring), ...","[(uh, uh, uh), (uh, uh, got), (uh, got, bring)..."


In [46]:
def get_common_unique_words(text, threshold=5):
    """
    Get common unique words in dataframe, aka words that occur in multiple readme's
    a word must appear in at least threshold readmes to be considered a common word
    """
    #splitting text into individual words
    words = text.split()
    #using collections to get an ngrams count 
    counter = Counter(words)
    #Create a list with all of the unique words using our default threshold of at least 5
    common_unique_words = [
        word for word, count in counter.items() if count >= threshold
    ]
    #Join the list with a new string
    new_string = " ".join(common_unique_words)
    return new_string

In [47]:
def n_most_common_word(string, n=1):
    """
    Return the nth most common word in a string
    """
    # Create a list of words separated by a space
    words = string.split()
    # Make an if statement that will only show the nth most common word based on the value set for n
    if len(words) < n:
        return ""
    # Use collections to get an ngram count
    word_counts = Counter(words)
    # Return only the most common
    return word_counts.most_common(n)[n - 1][0]

In [48]:
df = prep.get_topics(df)
df.head(3)

topics added                        


,title,artist,date,lyrics,raw_lyrics,decade,character_count,word_count,unique_words,unique_words_count,sentiment,sentiment_category,place_words,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count,bigrams,trigrams,topic_name
0,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,#1 LyricsUh uh uh I just gotta bring it to the...,2000,2014,363,learn hiphop uhh album aiyyo mean catch right ...,228,0.9915,very positive,[hook] [hook] [hook],0,0,0.0,0,0,0,3,"[(uh, uh), (uh, uh), (uh, got), (got, bring), ...","[(uh, uh, uh), (uh, uh, got), (uh, got, bring)...",sex
1,#9 Dream,John Lennon,1974-12-21,long ago dream dream know yes know seemed real...,#9 Dream Lyrics[Verse 1] So long ago Was it in...,1970,842,135,dancing long call air feel bowakawa go dance s...,49,0.9169,very positive,[verse 1] [pre-chorus 1] [chorus] [verse 2] [p...,2,2,1.0,2,1,0,0,"[(long, ago), (ago, dream), (dream, dream), (d...","[(long, ago, dream), (ago, dream, dream), (dre...",love
2,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,ah ah beautiful ah ah beautiful hop back bike ...,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,...",2010,768,129,feel beautiful oh mean stop dance yes yeah nig...,54,0.9989,very positive,[intro: mariah carey] [verse 1: miguel] [choru...,2,2,1.0,0,1,0,0,"[(ah, ah), (ah, beautiful), (beautiful, ah), (...","[(ah, ah, beautiful), (ah, beautiful, ah), (be...",lost


In [49]:
df.topic_name.value_counts()

breakups          4831
lost              3237
affection         2815
sex               1790
nature            1756
love              1735
good vibes        1284
americana         1211
youth             1116
violence          1066
money              672
dance              653
transcendental     550
heartache          339
jealousy           259
holiday            227
spanish            221
Name: topic_name, dtype: int64

In [50]:
# combine all strings in lyrics where decade is the same
decades = (
    df.groupby("decade")["lyrics"].apply(lambda x: " ".join(x)).reset_index()
)
decades.rename(columns={"lyrics": "all_words"}, inplace=True)
decades["unique_words"] = (
    df.groupby("decade")["unique_words"].apply(lambda x: " ".join(x)).values
)
decades["common_unique_words"] = decades.unique_words.apply(get_common_unique_words)
decades["n_words"] = decades["all_words"].apply(lambda x: len(x.split()))
decades["unique_word_count"] = decades["all_words"].apply(
    lambda x: len(set(x.split()))
)
decades["mean_word_count"] = (
    df.groupby("decade")["word_count"].mean().values.round(1)
)
decades["median_word_count"] = (
    df.groupby("decade")["word_count"].median().values.round(1)
)
decades["most_common_word"] = decades["unique_words"].apply(
    lambda x: n_most_common_word(x)
)
decades["2nd_most_common_word"] = decades["unique_words"].apply(
    lambda x: n_most_common_word(x, 2)
)
decades["3rd_most_common_word"] = decades["unique_words"].apply(
    lambda x: n_most_common_word(x, 3)
)
decades["4th_most_common_word"] = decades["unique_words"].apply(
    lambda x: n_most_common_word(x, 4)
)
decades["5th_most_common_word"] = decades["unique_words"].apply(
    lambda x: n_most_common_word(x, 5)
)



In [51]:
decades.head(10)

,decade,all_words,unique_words,common_unique_words,n_words,unique_word_count,mean_word_count,median_word_count,most_common_word,2nd_most_common_word,3rd_most_common_word,4th_most_common_word,5th_most_common_word
0,1950,never felt like kissed ya exist kissed ya neve...,oh live yeah til way life mind realize since m...,oh live yeah til way life mind realize since m...,43809,4697,84.7,72.0,love,know,heart,come,oh
1,1960,till moon desert sky till sea run dry till wor...,grows upstream oh give live flow dream desert ...,grows oh give live flow dream desert river run...,414717,14120,90.8,85.0,love,know,oh,got,go
2,1970,long ago dream dream know yes know seemed real...,dancing long call air feel bowakawa go dance s...,dancing long call air feel go dance strange th...,457047,14059,113.9,105.0,love,know,got,oh,like
3,1980,car hop bebop sang wop diddy wop diddy wop doo...,oh doo nowhere go give woo end yeah bad night ...,oh doo nowhere go give woo end yeah bad night ...,451583,13486,125.2,117.0,know,love,time,got,like
4,1990,love gave heart best keep satisfied took love ...,feel someone find inside give cruel deny shine...,feel someone find inside give cruel deny shine...,492327,19391,172.4,144.0,know,love,like,want,time
5,2000,uh uh uh got bring attention dirty better watc...,learn hiphop uhh album aiyyo mean catch right ...,learn hiphop uhh album aiyyo mean catch right ...,616267,21487,194.4,162.5,know,like,got,want,get
6,2010,ah ah beautiful ah ah beautiful hop back bike ...,feel beautiful oh mean stop dance yes yeah nig...,feel beautiful oh mean stop dance yes yeah nig...,819659,26068,205.0,184.0,like,know,got,get,want
7,2020,tell old man largemouth fishin another time go...,may dream rain show chanc somebody ask wait br...,may dream rain show somebody ask wait bring ch...,235861,14865,227.9,211.0,like,got,know,get,yeah


In [52]:
decades.decade.unique()

array([1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020])

In [53]:
# #add columns to decades dataframe for topic name counts
# for topic in df.topic_name.unique():
#     decades[f'{topic}_count'] = decades.decade.apply(lambda x: df[(df.decade == x) & (df.topic_name == topic)].shape[0])
#     decades[f'{topic}_percent'] = decades.decade.apply(lambda x: df[(df.decade == x) & (df.topic_name == topic)].shape[0]/df[df.decade == x].shape[0] *100)# 

In [54]:
# # make a list of all the columns that have percent in the name
# percent_cols = [col for col in decades.columns if "percent" in col]
# decades_percent = decades[percent_cols]
# decades_percent.head(2)

# feature engineering modeling

In [55]:
# get fresh df
df = prep.get_data()
# add topics
df = prep.get_topics(df)

#drop incomplete decades for modeling
# remove incomplete decades (1950, 2020)
df = df[(df.decade != 1950) & (df.decade != 2020)]

Features added ******************
topics added                        


In [56]:
df.head()

,title,artist,date,lyrics,raw_lyrics,decade,character_count,word_count,unique_words,unique_words_count,sentiment,sentiment_category,place_words,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count,bigrams,trigrams,topic_name
0,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,#1 LyricsUh uh uh I just gotta bring it to the...,2000,2014,363,learn hiphop uhh album aiyyo mean catch right ...,228,0.9915,very positive,[hook] [hook] [hook],0,0,0.000000,0,0,0,3,"[(uh, uh), (uh, uh), (uh, got), (got, bring), ...","[(uh, uh, uh), (uh, uh, got), (uh, got, bring)...",sex
1,#9 Dream,John Lennon,1974-12-21,long ago dream dream know yes know seemed real...,#9 Dream Lyrics[Verse 1] So long ago Was it in...,1970,842,135,dancing long call air feel bowakawa go dance s...,49,0.9169,very positive,[verse 1] [pre-chorus 1] [chorus] [verse 2] [p...,2,2,1.000000,2,1,0,0,"[(long, ago), (ago, dream), (dream, dream), (d...","[(long, ago, dream), (ago, dream, dream), (dre...",love
2,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,ah ah beautiful ah ah beautiful hop back bike ...,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,...",2010,768,129,feel beautiful oh mean stop dance yes yeah nig...,54,0.9989,very positive,[intro: mariah carey] [verse 1: miguel] [choru...,2,2,1.000000,0,1,0,0,"[(ah, ah), (ah, beautiful), (beautiful, ah), (...","[(ah, ah, beautiful), (ah, beautiful, ah), (be...",lost
3,#SELFIE,The Chainsmokers,2014-03-15,jason table kept seeing look girl think make j...,#SELFIE Lyrics[Verse 1] When Jason was at the ...,2010,954,172,feel dance first shot valencia throw tacky wai...,100,0.9218,very positive,[verse 1] [drop 1] [verse 2] [drop 2] [verse 3...,0,3,0.000000,0,0,0,0,"[(jason, table), (table, kept), (kept, seeing)...","[(jason, table, kept), (table, kept, seeing), ...",violence
4,#thatPOWER,will.i.am Featuring Justin Bieber,2013-04-06,oh alive alive alive oh fly fly fly oh alive a...,#thatPOWER Lyrics[Instrumental break] [Pre-Ch...,2010,1238,208,high fresh fire chest bank yyyyes hatin burnin...,77,0.9984,very positive,[instrumental break] [pre-chorus: justin biebe...,3,2,0.666667,3,1,1,0,"[(oh, alive), (alive, alive), (alive, alive), ...","[(oh, alive, alive), (alive, alive, alive), (a...",sex


In [57]:
# split data into train validate and test
train, validate, test = prep.split_data(df)

In [58]:
def get_common_unique_words(text, threshold=5):
    """
    Get common unique words in dataframe, aka words that occur in multiple readme's
    a word must appear in at least threshold readmes to be considered a common word
    """

    words = text.split()
    counter = Counter(words)
    common_unique_words = [
        word for word, count in counter.items() if count >= threshold
    ]
    new_string = " ".join(common_unique_words)
    return new_string


In [59]:

def n_most_common_word(string, n=1):
    """
    Return the nth most common word in a string
    """
    # Create a list of words separated by a space
    words = string.split()
    # Make an if statement that will only show the nth most common word based on the value set for n
    if len(words) < n:
        return ""
    # Use collections to get an ngram count
    word_counts = Counter(words)
    # Return only the most common
    return word_counts.most_common(n)[n - 1][0]


In [60]:
# combine all strings in lyrics where decade is the same
decades = train.groupby("decade")["lyrics"].apply(lambda x: " ".join(x)).reset_index()
decades.rename(columns={"lyrics": "all_words"}, inplace=True)
decades["unique_words"] = (
    train.groupby("decade")["unique_words"].apply(lambda x: " ".join(x)).values
)
decades["common_unique_words"] = decades.unique_words.apply(get_common_unique_words)
decades["n_words"] = decades["all_words"].apply(lambda x: len(x.split()))
decades["unique_word_count"] = decades["all_words"].apply(lambda x: len(set(x.split())))
decades["mean_word_count"] = (
    train.groupby("decade")["word_count"].mean().values.round(1)
)
decades["median_word_count"] = (
    train.groupby("decade")["word_count"].median().values.round(1)
)
decades["most_common_word"] = decades["unique_words"].apply(
    lambda x: n_most_common_word(x)
)
decades["2nd_most_common_word"] = decades["unique_words"].apply(
    lambda x: n_most_common_word(x, 2)
)
decades["3rd_most_common_word"] = decades["unique_words"].apply(
    lambda x: n_most_common_word(x, 3)
)
decades["4th_most_common_word"] = decades["unique_words"].apply(
    lambda x: n_most_common_word(x, 4)
)
decades["5th_most_common_word"] = decades["unique_words"].apply(
    lambda x: n_most_common_word(x, 5)
)



In [61]:
decades

,decade,all_words,unique_words,common_unique_words,n_words,unique_word_count,mean_word_count,median_word_count,most_common_word,2nd_most_common_word,3rd_most_common_word,4th_most_common_word,5th_most_common_word
0,1960,hey girl want know going miss much go hey girl...,long inside go miss live die deep want know su...,long inside go miss live die deep want know su...,232224,10340,90.8,85.0,love,know,oh,go,got
1,1970,walking sidewalk purple afternoon accosted bar...,wearing sidewalk imaginary someone oh crawling...,wearing sidewalk someone oh go upon holding co...,256399,10518,114.2,105.0,love,know,got,oh,like
2,1980,get high horse let go one one make feel real y...,stuff feel high oh go yeah cared plan fan else...,stuff feel high oh go yeah cared plan fan else...,250920,9522,124.3,117.0,know,love,time,got,oh
3,1990,want yeah let hear say warren g warren g want ...,skirt dipped high head fat hennessey thought l...,skirt dipped high head fat hennessey thought l...,279460,14357,174.7,146.0,know,love,like,time,want
4,2000,shawty shawty shawty yeah yeah get lonely busi...,find someone oh give sitting go cash little me...,find someone oh give sitting go cash little me...,338454,15686,190.7,161.0,know,like,got,want,get
5,2010,mmm mmm oh yeah oh yeah yeah alright save need...,close stay saving guard feel oh call mean nowh...,close stay saving guard feel oh call mean nowh...,463311,19705,206.9,185.0,like,know,got,get,want


In [62]:

def get_common_bigrams(text, threshold=5):
    """
    This function takes in a text and returns a list of the top 5 bigrams that are common to the text.
    """
    # get all bigrams in text
    bigrams = pd.Series(nltk.ngrams(text.split(), 2)).value_counts()
    # filter out bigrams that are less than threshold
    bigrams = bigrams[bigrams > threshold]
    # return all that occur more than 5 times
    return bigrams.index.values



In [63]:

decades["common_bigrams"] = decades.all_words.apply(lambda x: get_common_bigrams(x))


# add bigrams to decade dataframe
decades["bigrams"] = decades.all_words.apply(
    lambda x: pd.Series(nltk.ngrams(x.split(), 2)).values
)

# makes sets of all common bigrams in all decades so no bigrams are repeated
set_60s = set(decades[decades.decade == 1960].common_bigrams.values[0])
set_70s = set(decades[decades.decade == 1970].common_bigrams.values[0])
set_80s = set(decades[decades.decade == 1980].common_bigrams.values[0])
set_90s = set(decades[decades.decade == 1990].common_bigrams.values[0])
set_00s = set(decades[decades.decade == 2000].common_bigrams.values[0])
set_10s = set(decades[decades.decade == 2010].common_bigrams.values[0])
# remove words found in other decades
unique_to_60 = set_60s-set_70s-set_80s-set_90s-set_00s-set_10s
unique_to_70 = set_70s-set_60s-set_80s-set_90s-set_00s-set_10s
unique_to_80 = set_80s-set_60s-set_70s-set_90s-set_00s-set_10s
unique_to_90 = set_90s-set_60s-set_70s-set_80s-set_00s-set_10s
unique_to_00 = set_00s-set_60s-set_70s-set_80s-set_90s-set_10s
unique_to_10 = set_10s-set_60s-set_70s-set_80s-set_90s-set_00s
# make a series to add to the dataframe
unique_to_decade = [unique_to_60, unique_to_70, unique_to_80, unique_to_90, unique_to_00, unique_to_10]
decades["bigrams_unique_to_decade"] = list(unique_to_decade)



In [64]:
decades.head(3)

,decade,all_words,unique_words,common_unique_words,n_words,unique_word_count,mean_word_count,median_word_count,most_common_word,2nd_most_common_word,3rd_most_common_word,4th_most_common_word,5th_most_common_word,common_bigrams,bigrams,bigrams_unique_to_decade
0,1960,hey girl want know going miss much go hey girl...,long inside go miss live die deep want know su...,long inside go miss live die deep want know su...,232224,10340,90.8,85.0,love,know,oh,go,got,"[(yeah, yeah), (la, la), (oh, oh), (let, u), (...","[(hey, girl), (girl, want), (want, know), (kno...","{(around, leaf), (cryin, eye), (purple, haze),..."
1,1970,walking sidewalk purple afternoon accosted bar...,wearing sidewalk imaginary someone oh crawling...,wearing sidewalk someone oh go upon holding co...,256399,10518,114.2,105.0,love,know,got,oh,like,"[(la, la), (yeah, yeah), (love, love), (oh, oh...","[(walking, sidewalk), (sidewalk, purple), (pur...","{(lied, would), (flatter, flatter), (jones, mr..."
2,1980,get high horse let go one one make feel real y...,stuff feel high oh go yeah cared plan fan else...,stuff feel high oh go yeah cared plan fan else...,250920,9522,124.3,117.0,know,love,time,got,oh,"[(oh, oh), (yeah, yeah), (let, u), (love, love...","[(get, high), (high, horse), (horse, let), (le...","{(matter, take), (beat, come), (glad, happy), ..."


In [65]:
len(decades.bigrams_unique_to_decade.values[0])

2588

In [66]:
# print len of bigrams_unique_to_decade for all decades
for  i in range(len(decades)):
    print(len(decades.bigrams_unique_to_decade[i])) 


2588
3043
3267
3264
4620
7299


In [67]:

def bigram_count(bigrams, decade):
    """
    This function takes in a list of bigrams and returns the count of bigrams that are in the decades unique set.
    """
    if decade == 1960:
        unique_set = unique_to_60
    elif decade == 1970:
        unique_set = unique_to_70
    elif decade == 1980:
        unique_set = unique_to_80
    elif decade == 1990:
        unique_set = unique_to_90
    elif decade == 2000:
        unique_set = unique_to_00
    elif decade == 2010:
        unique_set = unique_to_10
    else:
        print("Error: decade not found")

    # count the number of bigrams that are in the unique set
    count = 0
    for bigram in bigrams:
        if bigram in unique_set:
            count += 1
    return count


In [68]:


def baseline_accuracy2(series, mode):
    """
    Calculate baseline accuracy
    """
    test = pd.DataFrame(series)
    test["mode"] = mode
    baseline_accuracy = accuracy_score(test["decade"], test["mode"])
    return baseline_accuracy


In [69]:
df.head()

,title,artist,date,lyrics,raw_lyrics,decade,character_count,word_count,unique_words,unique_words_count,sentiment,sentiment_category,place_words,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count,bigrams,trigrams,topic_name
0,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,#1 LyricsUh uh uh I just gotta bring it to the...,2000,2014,363,learn hiphop uhh album aiyyo mean catch right ...,228,0.9915,very positive,[hook] [hook] [hook],0,0,0.000000,0,0,0,3,"[(uh, uh), (uh, uh), (uh, got), (got, bring), ...","[(uh, uh, uh), (uh, uh, got), (uh, got, bring)...",sex
1,#9 Dream,John Lennon,1974-12-21,long ago dream dream know yes know seemed real...,#9 Dream Lyrics[Verse 1] So long ago Was it in...,1970,842,135,dancing long call air feel bowakawa go dance s...,49,0.9169,very positive,[verse 1] [pre-chorus 1] [chorus] [verse 2] [p...,2,2,1.000000,2,1,0,0,"[(long, ago), (ago, dream), (dream, dream), (d...","[(long, ago, dream), (ago, dream, dream), (dre...",love
2,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,ah ah beautiful ah ah beautiful hop back bike ...,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,...",2010,768,129,feel beautiful oh mean stop dance yes yeah nig...,54,0.9989,very positive,[intro: mariah carey] [verse 1: miguel] [choru...,2,2,1.000000,0,1,0,0,"[(ah, ah), (ah, beautiful), (beautiful, ah), (...","[(ah, ah, beautiful), (ah, beautiful, ah), (be...",lost
3,#SELFIE,The Chainsmokers,2014-03-15,jason table kept seeing look girl think make j...,#SELFIE Lyrics[Verse 1] When Jason was at the ...,2010,954,172,feel dance first shot valencia throw tacky wai...,100,0.9218,very positive,[verse 1] [drop 1] [verse 2] [drop 2] [verse 3...,0,3,0.000000,0,0,0,0,"[(jason, table), (table, kept), (kept, seeing)...","[(jason, table, kept), (table, kept, seeing), ...",violence
4,#thatPOWER,will.i.am Featuring Justin Bieber,2013-04-06,oh alive alive alive oh fly fly fly oh alive a...,#thatPOWER Lyrics[Instrumental break] [Pre-Ch...,2010,1238,208,high fresh fire chest bank yyyyes hatin burnin...,77,0.9984,very positive,[instrumental break] [pre-chorus: justin biebe...,3,2,0.666667,3,1,1,0,"[(oh, alive), (alive, alive), (alive, alive), ...","[(oh, alive, alive), (alive, alive, alive), (a...",sex


In [70]:
# df["bigrams_from_60s"] = df.bigrams.apply(lambda x: bigram_count(x, 1960))
# df["bigrams_from_70s"] = df.bigrams.apply(lambda x: bigram_count(x, 1970))
# df["bigrams_from_80s"] = df.bigrams.apply(lambda x: bigram_count(x, 1980))
# df["bigrams_from_90s"] = df.bigrams.apply(lambda x: bigram_count(x, 1990))
# df["bigrams_from_00s"] = df.bigrams.apply(lambda x: bigram_count(x, 2000))
# df["bigrams_from_10s"] = df.bigrams.apply(lambda x: bigram_count(x, 2010))

In [71]:
df.isnull().sum()

title                 0
artist                0
date                  0
lyrics                0
raw_lyrics            0
decade                0
character_count       0
word_count            0
unique_words          0
unique_words_count    0
sentiment             0
sentiment_category    0
place_words           0
chorus_count          0
verse_count           0
verse_chorus_ratio    0
pre_chorus_count      0
outro_count           0
bridge_count          0
hook_count            0
bigrams               0
trigrams              0
topic_name            0
dtype: int64

In [72]:
df.describe()

,decade,character_count,word_count,unique_words_count,sentiment,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count
count,22210.000000,22210.000000,22210.000000,22210.000000,22210.000000,22210.000000,22210.000000,22210.000000,22210.000000,22210.000000,22210.000000,22210.000000
mean,1983.622692,838.231968,146.402521,68.675236,0.536993,1.553534,1.490545,0.461749,0.314498,0.311436,0.337371,0.135119
std,17.866680,493.384062,86.837353,45.273977,0.735576,1.614286,1.452577,0.553325,0.801475,0.467253,0.568076,0.692989
min,1960.000000,0.000000,0.000000,0.000000,-0.999900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1970.000000,512.000000,89.000000,42.000000,0.421500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1980.000000,710.000000,123.000000,56.000000,0.964800,2.000000,2.000000,0.400000,0.000000,0.000000,0.000000,0.000000
75%,2000.000000,1004.000000,175.000000,77.000000,0.992300,3.000000,2.000000,0.750000,0.000000,1.000000,1.000000,0.000000
max,2010.000000,10282.000000,1757.000000,906.000000,0.999900,11.000000,16.000000,13.000000,6.000000,3.000000,6.000000,9.000000


In [73]:
numeric_cols = [
    col for col in train.columns if train[col].dtype in ["int64", "float64"]
]
numeric_cols

['decade',
 'character_count',
 'word_count',
 'unique_words_count',
 'sentiment',
 'chorus_count',
 'verse_count',
 'verse_chorus_ratio',
 'pre_chorus_count',
 'outro_count',
 'bridge_count',
 'hook_count']

In [104]:


###############################################################################
#                                                                             #
#               run models on feature engineered columns                      #
#                                                                             #
###############################################################################


def run_models_on_feature_engineered_columns(df, scale=True, bigrams_only=False):
    """
    Run models on data varying solver and C value
    """
    # # add bigram counts to main df
    # df["bigrams"] = df.lyrics.apply(
    #     lambda x: pd.Series(nltk.ngrams(x.split(), 2)).values
    #)
    df["bigrams_from_60s"] = df.bigrams.apply(lambda x: bigram_count(x, 1960))
    df["bigrams_from_70s"] = df.bigrams.apply(lambda x: bigram_count(x, 1970))
    df["bigrams_from_80s"] = df.bigrams.apply(lambda x: bigram_count(x, 1980))
    df["bigrams_from_90s"] = df.bigrams.apply(lambda x: bigram_count(x, 1990))
    df["bigrams_from_00s"] = df.bigrams.apply(lambda x: bigram_count(x, 2000))
    df["bigrams_from_10s"] = df.bigrams.apply(lambda x: bigram_count(x, 2010))

    # split data into train, validate, and test sets
    train, validate, test = prep.split_data(df)
    y_train = train.decade
    y_validate = validate.decade
    y_test = test.decade
    # list all columns with dtype int or float
    numeric_cols = [
        col for col in train.columns if train[col].dtype in ["int64", "float64"]
    ]
    # keep only numeric columns
    X_train = train[numeric_cols].drop(['decade'], axis=1)
    X_validate = validate[numeric_cols].drop(['decade'], axis=1)
    X_test = test[numeric_cols].drop(['decade'], axis=1)
    if bigrams_only:
        bigram_columns = [col for col in train.columns if "_bigrams" in col]
        X_train = train[bigram_columns]
        X_validate = validate[bigram_columns]
        X_test = test[bigram_columns]
    if scale:
        print("Scaling data")
        # scale data
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_validate = scaler.transform(X_validate)
        X_test = scaler.transform(X_test)

    # get mode to use as baseline
    mode = df.decade.mode().values[0]
    # get baseline_accuracy
    train_baseline = baseline_accuracy2(y_train, mode)
    validate_baseline = baseline_accuracy2(y_validate, mode)
    test_baseline = baseline_accuracy2(y_test, mode)
    # make a df for results
    results = pd.DataFrame()
    # make baseline model
    baseline_model = pd.Series(
        {
            "model_number": "baseline",
            "model_type": "baseline",
            "train_accuracy": train_baseline,
            "validate_accuracy": validate_baseline,
            "test_accuracy": test_baseline,
            "better_than_baseline": False,
        }
    )
    # add baseline model to results df
    results = pd.concat([results, baseline_model], axis=0)
    # make more models varying solver
    model_number = results.shape[1]
    c_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    for solver in ["liblinear", "lbfgs", "newton-cg", "sag", "saga"]:
        for c in c_values:
            # make the model
            lm = LogisticRegression(C=c, solver=solver).fit(X_train, y_train)
            # run model on data splits
            train["predicted"] = lm.predict(X_train)
            validate["predicted"] = lm.predict(X_validate)
            test["predicted"] = lm.predict(X_test)
            # make results series to add to results df
            stats = pd.Series(
                {
                    "model_number": model_number,
                    "model_type": "LogisticRegression",
                    "solver": solver,
                    "C": c,
                    "train_accuracy": accuracy_score(y_train, train["predicted"]),
                    "validate_accuracy": accuracy_score(
                        y_validate, validate["predicted"]
                    ),
                    "test_accuracy": accuracy_score(y_test, test["predicted"]),
                    "baseline_accuracy": validate_baseline,
                    "better_than_baseline": accuracy_score(
                        y_validate, validate["predicted"]
                    )
                    > validate_baseline,
                }
            )
            # add to results df
            results = pd.concat([results, stats], axis=1)
            model_number += 1

    return results.T.reset_index(drop=True)


In [90]:
results = run_models_on_feature_engineered_columns(df, scale=False)

In [91]:
results.sort_values(['validate_accuracy','test_accuracy'], ascending=False).head(3)

,model_number,model_type,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline,solver,C,baseline_accuracy
17,17,LogisticRegression,0.370829,0.36466,0.366727,True,newton-cg,0.1,0.20559
3,3,LogisticRegression,0.36713,0.36466,0.361099,True,liblinear,0.1,0.20559
18,18,LogisticRegression,0.370749,0.364284,0.366276,True,newton-cg,1,0.20559


In [105]:
results2 = run_models_on_feature_engineered_columns(df)

Scaling data


In [97]:
results2.sort_values(['validate_accuracy','test_accuracy'], ascending=False)

,model_number,model_type,train_accuracy,validate_accuracy,test_accuracy,better_than_baseline,solver,C,baseline_accuracy
18,18,LogisticRegression,0.746723,0.375914,0.377533,True,newton-cg,1,0.20559
32,32,LogisticRegression,0.746723,0.375914,0.377533,True,saga,1,0.20559
25,25,LogisticRegression,0.746884,0.375539,0.377983,True,sag,1,0.20559
11,11,LogisticRegression,0.74608,0.374977,0.379109,True,lbfgs,1,0.20559
4,4,LogisticRegression,0.743186,0.371975,0.37258,True,liblinear,1,0.20559
33,33,LogisticRegression,0.792394,0.363909,0.368753,True,saga,10,0.20559
19,19,LogisticRegression,0.792233,0.363534,0.368978,True,newton-cg,10,0.20559
26,26,LogisticRegression,0.792394,0.363534,0.368978,True,sag,10,0.20559
12,12,LogisticRegression,0.788454,0.362784,0.367627,True,lbfgs,10,0.20559
5,5,LogisticRegression,0.791027,0.362221,0.364475,True,liblinear,10,0.20559


In [106]:
df["bigrams_from_60s"] = df.bigrams.apply(lambda x: bigram_count(x, 1960))
df["bigrams_from_70s"] = df.bigrams.apply(lambda x: bigram_count(x, 1970))
df["bigrams_from_80s"] = df.bigrams.apply(lambda x: bigram_count(x, 1980))
df["bigrams_from_90s"] = df.bigrams.apply(lambda x: bigram_count(x, 1990))
df["bigrams_from_00s"] = df.bigrams.apply(lambda x: bigram_count(x, 2000))
df["bigrams_from_10s"] = df.bigrams.apply(lambda x: bigram_count(x, 2010))
# split data into train, validate, and test sets
train, validate, test = prep.split_data(df)
y_train = train.decade
y_validate = validate.decade
y_test = test.decade
# list all columns with dtype int or float
numeric_cols = [
    col for col in train.columns if train[col].dtype in ["int64", "float64"]
]
# keep only numeric columns
X_train = train[numeric_cols].drop(['decade'], axis=1)
X_validate = validate[numeric_cols]
X_test = test[numeric_cols]

In [107]:
X_train

,character_count,word_count,unique_words_count,sentiment,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count,bigrams_from_60s,bigrams_from_70s,bigrams_from_80s,bigrams_from_90s,bigrams_from_00s,bigrams_from_10s
11947,648,97,57,-0.8591,2,2,1.000000,0,1,0,0,0,1,0,0,1,0
17460,1570,264,156,0.9971,3,3,1.000000,0,0,0,0,0,1,0,76,2,0
17464,885,164,74,0.9786,3,2,0.666667,2,1,0,0,0,0,2,5,0,2
8924,1182,223,75,0.9961,3,2,0.666667,2,1,1,0,5,2,0,2,10,1
7759,239,46,32,-0.8635,0,0,0.000000,0,0,0,0,1,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11685,685,122,57,0.9937,3,3,1.000000,0,1,0,0,2,0,17,0,0,1
9629,1796,298,132,-0.9971,3,3,1.000000,0,1,0,0,0,2,4,0,0,58
8988,329,60,24,0.9911,0,3,0.000000,0,0,2,0,2,0,0,0,0,2
1526,762,129,64,0.9990,3,2,0.666667,2,1,2,0,0,3,1,3,2,1


In [115]:
def get_data(file="songs_0526.csv", use_cache = True):
    """
    Get data from csv file,
    clean the data,
    add features
    return df
    """
    filename = "preppred_songs.csv"
    if os.path.isfile(filename) and use_cache:
        print("Reading from csv...")
        return pd.read_csv(filename)
    # read csv file
    print("reading csv file...", end="\r")
    df = pd.read_csv(file)
    # clean data
    df = prep.clean_df(df)
    # add features
    df = prep.add_features(df)
    # add topics
    df = prep.get_topics(df)
    print("Saving to csv in local directory...")
    df.to_csv(filename, index=False)
    return df

In [2]:
df=prep.get_data(use_cache=False)


Features added ******************
topics added                        
Saving to csv in local directory...


In [3]:
df = prep.get_data()
df

Reading from csv...


,title,artist,date,lyrics,raw_lyrics,decade,character_count,word_count,unique_words,unique_words_count,...,chorus_count,verse_count,verse_chorus_ratio,pre_chorus_count,outro_count,bridge_count,hook_count,bigrams,trigrams,topic_name
0,#1,Nelly,2001-10-20,uh uh uh got bring attention dirty better watc...,#1 LyricsUh uh uh I just gotta bring it to the...,2000,2014,363,word erything chat pant man eh rapper want hal...,228,...,0,0,0.000000,0,0,0,3,"[('uh', 'uh'), ('uh', 'uh'), ('uh', 'got'), ('...","[('uh', 'uh', 'uh'), ('uh', 'uh', 'got'), ('uh...",sex
1,#9 Dream,John Lennon,1974-12-21,long ago dream dream know yes know seemed real...,#9 Dream Lyrics[Verse 1] So long ago Was it in...,1970,842,135,warm cold feel whispered strange tree hear dan...,49,...,2,2,1.000000,2,1,0,0,"[('long', 'ago'), ('ago', 'dream'), ('dream', ...","[('long', 'ago', 'dream'), ('ago', 'dream', 'd...",love
2,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,ah ah beautiful ah ah beautiful hop back bike ...,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,...",2010,768,129,stop feel wind dance lord moonlight thing ligh...,54,...,2,2,1.000000,0,1,0,0,"[('ah', 'ah'), ('ah', 'beautiful'), ('beautifu...","[('ah', 'ah', 'beautiful'), ('ah', 'beautiful'...",lost
3,#SELFIE,The Chainsmokers,2014-03-15,jason table kept seeing look girl think make j...,#SELFIE Lyrics[Verse 1] When Jason was at the ...,2010,954,172,filter want smoke see valencia okay oh pause g...,100,...,0,3,0.000000,0,0,0,0,"[('jason', 'table'), ('table', 'kept'), ('kept...","[('jason', 'table', 'kept'), ('table', 'kept',...",violence
4,#thatPOWER,will.i.am Featuring Justin Bieber,2013-04-06,oh alive alive alive oh fly fly fly oh alive a...,#thatPOWER Lyrics[Instrumental break] [Pre-Ch...,2010,1238,208,bigger bank oh good hustle used alive door lov...,77,...,3,2,0.666667,3,1,1,0,"[('oh', 'alive'), ('alive', 'alive'), ('alive'...","[('oh', 'alive', 'alive'), ('alive', 'alive', ...",sex
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23757,Which Way You Goin' Billy?,The Poppy Family (Featuring Susan Jacks),1970-03-28,way going billy go way going billy go really l...,Which Way You Goin’ Billy? LyricsWhich way you...,1970,535,98,go soul want whole would thought billy show lo...,36,...,0,0,0.000000,0,0,0,0,"[('way', 'going'), ('going', 'billy'), ('billy...","[('way', 'going', 'billy'), ('going', 'billy',...",affection
23758,White Christmas,The Drifters Featuring Clyde McPhatter And Bil...,1960-12-19,ooh doop doop doop doo doop ooh doop doop doop...,"White Christmas LyricsOoh Doop doop, doop doo ...",1960,526,90,hear ooooh ooh sleigh treetop dreaming may jin...,29,...,0,0,0.000000,0,0,0,0,"[('ooh', 'doop'), ('doop', 'doop'), ('doop', '...","[('ooh', 'doop', 'doop'), ('doop', 'doop', 'do...",holiday
23759,Why Don't You & I,Santana Featuring Alex Band Or Chad Kroeger,2003-07-26,since moment spotted like walking round little...,Why Don’t You & I Lyrics[Verse 1: Chad Kroeger...,2000,1146,195,moon go walking butterfly man ooh hold heaven ...,65,...,4,2,0.500000,3,0,1,0,"[('since', 'moment'), ('moment', 'spotted'), (...","[('since', 'moment', 'spotted'), ('moment', 's...",love
23760,Wild Wild West,Will Smith Featuring Dru Hill & Kool Mo Dee,1999-05-22,mmmts uhh wickiwild wild wickiwickiwild wickiw...,Wild Wild West Lyrics[Intro: Will Smith & (Sis...,1990,2269,408,defender tree want man quickest see buffalo go...,170,...,1,3,3.000000,1,0,0,0,"[('mmmts', 'uhh'), ('uhh', 'wickiwild'), ('wic...","[('mmmts', 'uhh', 'wickiwild'), ('uhh', 'wicki...",lost
